In [107]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
import tiktoken

In [108]:
story_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Character: {input}\nStory: {output}",
)

In [109]:
examples = [
    {"input": "Gollum", "output": "<Story involving Gollum>"},
    {"input": "Gandalf", "output": "<Story involving Gandalf>"},
    {"input": "Bilbo", "output": "<Story involving Bilbo>"},
]

In [110]:
def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [111]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=story_prompt,
    max_length=40,
    # get_text_length: Callable[[str], int] = lambda x: len(re.split("\n| ", x))
    # You have modified the get_text_length function to work with the TikToken library based on token usage:
    get_text_length=num_tokens_from_string,
)

In [112]:
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=story_prompt,
    prefix="Generate a story for {character} using the current Character/Story pairs from all of the characters as context.",
    suffix="Character: {character}\nStory:",
    input_variables=["character"],
)

In [113]:
# Provide a new character from lord of the rings
formatted_prompt = dynamic_prompt.format(character="Frodo")

In [114]:
print(formatted_prompt)

Generate a story for Frodo using the current Character/Story pairs from all of the characters as context.

Character: Gollum
Story: <Story involving Gollum>

Character: Gandalf
Story: <Story involving Gandalf>

Character: Frodo
Story:


In [115]:
chat = ChatOpenAI()

response = chat([SystemMessage(content=formatted_prompt)])
print(response)

content="Frodo was a young hobbit living a peaceful life in the Shire. However, his life took an unexpected turn when his uncle, Bilbo Baggins, decided to leave behind his magical ring and go on an adventure.\n\nLittle did Frodo know, this ring was no ordinary trinket. It was the One Ring, a powerful artifact desired by the dark lord Sauron. Gandalf, a wise and powerful wizard, realized the danger this ring posed and sought to protect Frodo from its influence.\n\nWith Gandalf's guidance, Frodo embarked on a treacherous journey to destroy the ring in the fires of Mount Doom, the only place where it could be destroyed. Along the way, they encountered many perils and strange creatures, one of which was Gollum.\n\nGollum, a former hobbit corrupted by the power of the ring, became obsessed with finding it again. He relentlessly pursued Frodo and Gandalf, driven by his insatiable desire for the ring's power. However, Frodo and Gandalf were determined to keep the ring out of Gollum's grasp.\n